# Import der IVU Fzg Log Dateien

- erster Ansatz aus ChatGPT

In [ ]:
import re
import json
import pandas as pd
from datetime import datetime

In [ ]:
file_path = "2025-11-11_Umsteigehinweise_EVB_Beispiel.txt"

file_path = "/home/zvbn/python/ivu_fzg_log/NetPeerManager.2025-11-13.1.log"

data = []
pattern = re.compile(r"<K(.*?)>")

with open(file_path, encoding="latin1") as f:
    for line in f:
        match = pattern.search(line)
        if not match:
            continue

        payload = match.group(1)
        fields = payload.split("#")

        for i in range(len(fields) - 15):
            print(f"{i}, {len(fields)}, {fields[i]}")
            if re.match(r"^\d{5,6}$", fields[i]):  # Start einer Fahrt erwartet werden 5 oder 6 stellige IDs
                try:
                    stop_id = fields[i]
                    stop_name = fields[i + 1]
                    mode = fields[i + 2]  # BUS, Tram, etc.
                    line_no = fields[i + 6] if len(fields) > i + 6 else None

                    # Ziel im JSON-Feld
                    json_field = fields[i + 11] if len(fields) > i + 11 else ""
                    final_json = None
                    if "{" in json_field and "}" in json_field:
                        try:
                            final_json = json.loads(json_field)
                        except json.JSONDecodeError:
                            pass

                    dep_time = fields[i + 12] if len(fields) > i + 12 else None
                    arr_time = fields[i + 13] if len(fields) > i + 13 else None

                    data.append({
                        "StopID": stop_id,
                        "Haltestelle": stop_name,
                        "Verkehrsmittel": mode,
                        "Linie": line_no,
                        "Abfahrt": dep_time,
                        "Ankunft": arr_time,
                        "Ziel": final_json["Final"]["Text"] if final_json else None,
                    })
                except Exception:
                    continue

# In DataFrame konvertieren
df = pd.DataFrame(data)



In [ ]:
df[df.Verkehrsmittel.isin(["BUS", "Tram"])]['Linie'].value_counts().plot.barh()

In [ ]:
# Nur sinnvolle Einträge behalten
df = df.dropna(subset=["Haltestelle", "Abfahrt", "Ziel"])
df = df.drop_duplicates()

# Abfahrtszeit als Datetime-Objekt
df["Abfahrt_dt"] = pd.to_datetime(df["Abfahrt"], errors="coerce")
df = df.sort_values(by=["Haltestelle", "Abfahrt_dt"]).reset_index(drop=True)

# Umstiegserkennung + Umstiegszeit
df["Umstieg_möglich"] = False
df["Umstiegszeit_min"] = None

for stop in df["Haltestelle"].unique():
    sub = df[df["Haltestelle"] == stop].sort_values("Abfahrt_dt")
    for i in range(1, len(sub)):
        delta = (sub.iloc[i]["Abfahrt_dt"] - sub.iloc[i - 1]["Abfahrt_dt"]).total_seconds() / 60
        if 0 < delta <= 10:
            df.loc[sub.index[i], "Umstieg_möglich"] = True
            df.loc[sub.index[i - 1], "Umstieg_möglich"] = True
            df.loc[sub.index[i], "Umstiegszeit_min"] = round(delta, 1)
            df.loc[sub.index[i - 1], "Umstiegszeit_min"] = round(delta, 1)

# Aufräumen
df = df.drop(columns=["Abfahrt_dt"])

# Ergebnis speichern
df.to_csv("evb_parsed_umstieg_zeiten.csv", index=False, encoding="utf-8-sig")
print(df.head(15))
print("\n✅ Ergebnis gespeichert als 'evb_parsed_umstieg_zeiten.csv'")

In [ ]:
df.head()